# Boston housing price predictor (regression) using DNN

## Step 1: Prepare training and evaluation dataset, create FastEstimator Pipeline

Pipeline can take both data in memory and data in disk. In this example, we are going to use data in memory by loading data with tf.keras.datasets.boston_housing

In [123]:
import tensorflow as tf

(x_train, y_train), (x_eval, y_eval) = tf.keras.datasets.boston_housing.load_data()
print("train shape is {}".format(x_train.shape))
print("train value shape is {}".format(y_train.shape))
print("eval shape is {}".format(x_eval.shape))
print("eval value shape is {}".format(y_eval.shape))

train shape is (404, 13)
train value shape is (404,)
eval shape is (102, 13)
eval value shape is (102,)


The next step is to scale the inputs to the neural network. This is done by using a StandardScaler.

In [124]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_eval = scaler.transform(x_eval)

For in-memory data in Pipeline, the data format should be a nested dictionary like: {"mode1": {"feature1": numpy_array, "feature2": numpy_array, ...}, ...}. Each mode can be either train or eval, in our case, we have both train and eval. feature is the feature name, in our case, we have x and y.

In [125]:
data = {
        "train": {
            "x": x_train, "y": y_train
        },
        "eval": {
            "x": x_eval, "y": y_eval
        }
    }

Now we are ready to define Pipeline:

In [126]:
import fastestimator as fe

pipeline = fe.Pipeline(batch_size=32, data=data)

## Step 2: Prepare model, create FastEstimator Network

First, we have to define the network architecture in tf.keras.Model or tf.keras.Sequential. After defining the architecture, users are expected to feed the architecture definition and its associated model name, optimizer and loss name (default to be 'loss') to FEModel.

In [127]:
from tensorflow.keras import layers
from fastestimator.network.model import FEModel

def create_dnn():
    model = tf.keras.Sequential()

    model.add(layers.Dense(64, activation="relu", input_shape=(13,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation="linear"))

    return model

model = FEModel(model_def=create_dnn, model_name="dnn", optimizer="adam")

Now we are ready to define the Network: given with a batch data with key x and y, we have to work our way to loss with series of operators. ModelOp is an operator that contains a model.

In [128]:
from fastestimator.network.model import ModelOp
from fastestimator.network.loss import MeanSquaredError

network = fe.Network(
    ops=[ModelOp(inputs="x", model=model, outputs="y_pred"), MeanSquaredError(y_true="y", y_pred="y_pred")])

## Step 3: Configure training, create Estimator

During the training loop, we want to: 1) measure lowest loss for data data 2) save the model with lowest valdiation loss. Trace class is used for anything related to training loop, we will need to import the ModelSaver trace.

In [129]:
import tempfile
from fastestimator.estimator.trace import ModelSaver

model_dir = tempfile.mkdtemp()
traces = [ModelSaver(model_name="dnn", save_dir=model_dir, save_best=True)]

Now we can define the Estimator and specify the training configuation:

In [130]:
estimator = fe.Estimator(network=network, pipeline=pipeline, epochs=50, traces=traces)

## Start Training

In [131]:
estimator.fit()

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Start: step: 0; dnn_lr: 0.001; 
FastEstimator-Train: step: 0; loss: 667.34644; 
FastEstimator-ModelSaver: Saving model to /tmp/tmp3yb752cp/dnn_best_loss.h5
FastEstimator-Eval: step: 12; epoch: 0; loss: 533.3779; since_best_loss: 0; min_loss: 533.3779; 
FastEstimator-Eval: step: 24; epoch: 1; loss: 568.65344; since_best_loss: 1; min_loss: 533.3779; 
FastEstimator-Eval: step: 36; epoch: 2; loss: 554.3767; since_best_loss: 2; min_loss: 533.3779; 
FastEstimator-ModelSaver: Saving model to /tmp/tmp3yb752cp/dnn_best_loss.h5
FastEstimator-Eval: ste

## Inferencing

After training, the model is saved to a temporary folder. we can load the model from file and do inferencing on a sample image.

In [132]:
import os

model_path = os.path.join(model_dir, 'dnn_best_loss.h5')
trained_model = tf.keras.models.load_model(model_path, compile=False)

Randomly get one sample from validation set and compare the predicted value with model's prediction:

In [136]:
import numpy as np

selected_idx = np.random.randint(0, high=101)
print("test sample idx {}, ground truth: {}".format(selected_idx, y_eval[selected_idx]))

test_sample = np.expand_dims(x_eval[selected_idx], axis=0)

predicted_value = trained_model.predict(test_sample)
print("model predicted value is {}".format(predicted_value))

test sample idx 94, ground truth: 35.1
model predicted value is [[19.924725]]
